In [170]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import statistics as st

def get_df(day):
    file_name = f"./round3/prices_round_3_day_{day}.csv"
    return pd.read_csv(file_name, sep=';')

def get_product(df, product):
    return df[df['product'] == product].copy()

def get_first_two_dfs():
    first_df = get_df(0)
    second_df = get_df(1)
    second_df['timestamp'] = second_df['timestamp'] + 1000000
    return pd.concat([first_df, second_df])



In [171]:
class Product:
    ROCK = 'VOLCANIC_ROCK'
    VOUCHER_9500 = 'VOLCANIC_ROCK_VOUCHER_9500'
    VOUCHER_9750 = 'VOLCANIC_ROCK_VOUCHER_9750'
    VOUCHER_10000 = 'VOLCANIC_ROCK_VOUCHER_10000'
    VOUCHER_10250 = 'VOLCANIC_ROCK_VOUCHER_10250'
    VOUCHER_10500 = 'VOLCANIC_ROCK_VOUCHER_10500'

position_limits = {
    Product.ROCK: 400,
    Product.VOUCHER_9500: 200,
    Product.VOUCHER_9750: 200,
    Product.VOUCHER_10000: 200,
    Product.VOUCHER_10250: 200,
    Product.VOUCHER_10500: 200
}

In [172]:
# prices = pd.read_csv('round2/prices_round_2_day_0.csv', sep=';')
# trades = pd.read_csv('round2/trades_round_2_day_0_nn.csv', sep=';')
prices = pd.concat([pd.read_csv(f'round3/prices_round_3_day_{i}.csv', sep=';') for i in range(3)], ignore_index=True)
trades = pd.concat([pd.read_csv(f'round3/trades_round_3_day_{i}_nn.csv', sep=';') for i in range(3)], ignore_index=True)

In [173]:
prices['swmid'] = (prices['bid_price_1'] * prices['ask_volume_1'] + prices['ask_price_1'] * prices['bid_volume_1']) / (prices['ask_volume_1'] + prices['bid_volume_1'])

In [174]:
rock = prices[prices["product"] == Product.ROCK].reset_index(drop=True).copy()
voucher_9500 = prices[prices["product"] == Product.VOUCHER_9500].reset_index(drop=True).copy()
voucher_9750 = prices[prices["product"] == Product.VOUCHER_9750].reset_index(drop=True).copy()
voucher_10000 = prices[prices["product"] == Product.VOUCHER_10000].reset_index(drop=True).copy()
voucher_10250 = prices[prices["product"] == Product.VOUCHER_10250].reset_index(drop=True).copy()
voucher_10500 = prices[prices["product"] == Product.VOUCHER_10500].reset_index(drop=True).copy()

In [175]:
[rock, voucher_9500, voucher_9750, voucher_10000, voucher_10250, voucher_10500] = [df.rename(columns={'ask_price_1': 'ask_price', 'bid_price_1': 'bid_price', 'ask_volume_1': 'ask_volume', 'bid_volume_1': 'bid_volume'}) for df in [rock, voucher_9500, voucher_9750, voucher_10000, voucher_10250, voucher_10500]]

In [176]:
rock = rock.drop(columns=['product'], axis=1).rename(columns={col: col + '_rock' for col in rock.columns if col not in ['timestamp', 'day']})
voucher_9500 = voucher_9500.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher9500' for col in voucher_9500.columns if col not in ['timestamp', 'day']})
voucher_9750 = voucher_9750.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher9750' for col in voucher_9750.columns if col not in ['timestamp', 'day']})
voucher_10000 = voucher_10000.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher10000' for col in voucher_10000.columns if col not in ['timestamp', 'day']})
voucher_10250 = voucher_10250.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher10250' for col in voucher_10250.columns if col not in ['timestamp', 'day']})
voucher_10500 = voucher_10500.drop(columns=['product'], axis=1).rename(columns={col: col + '_voucher10500' for col in voucher_10500.columns if col not in ['timestamp', 'day']})

In [177]:
# join croissants, james, djembes, basket1, basket2 on timestamp
mk = rock.merge(voucher_9500, on=['day', 'timestamp'])
mk = mk.merge(voucher_9750, on=['day', 'timestamp'])
mk = mk.merge(voucher_10000, on=['day', 'timestamp'])
mk = mk.merge(voucher_10250, on=['day', 'timestamp'])
mk = mk.merge(voucher_10500, on=['day', 'timestamp'])
mk

,day,timestamp,bid_price_rock,bid_volume_rock,bid_price_2_rock,bid_volume_2_rock,bid_price_3_rock,bid_volume_3_rock,ask_price_rock,ask_volume_rock,...,bid_volume_3_voucher10500,ask_price_voucher10500,ask_volume_voucher10500,ask_price_2_voucher10500,ask_volume_2_voucher10500,ask_price_3_voucher10500,ask_volume_3_voucher10500,mid_price_voucher10500,profit_and_loss_voucher10500,swmid_voucher10500
0,0,0,10502.0,205.0,NaN,NaN,NaN,NaN,10504,146,...,NaN,100,19,NaN,NaN,NaN,NaN,99.5,0.0,99.5
1,0,100,10509.0,192.0,NaN,NaN,NaN,NaN,10511,122,...,NaN,103,13,NaN,NaN,NaN,NaN,102.5,0.0,102.5
2,0,200,10512.0,167.0,NaN,NaN,NaN,NaN,10514,109,...,NaN,105,15,NaN,NaN,NaN,NaN,104.5,0.0,104.5
3,0,300,10517.0,118.0,10516.0,68.0,NaN,NaN,10518,118,...,NaN,107,13,NaN,NaN,NaN,NaN,106.5,0.0,106.5
4,0,400,10509.0,128.0,10508.0,73.0,NaN,NaN,10510,128,...,NaN,103,15,NaN,NaN,NaN,NaN,102.5,0.0,102.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2,999500,10167.0,110.0,10166.0,51.0,NaN,NaN,10169,156,...,NaN,4,20,NaN,NaN,NaN,NaN,3.5,0.0,3.2
29996,2,999600,10166.0,161.0,NaN,NaN,NaN,NaN,10168,101,...,NaN,4,14,NaN,NaN,NaN,NaN,3.5,0.0,3.5
29997,2,999700,10166.0,143.0,10165.0,51.0,NaN,NaN,10168,194,...,NaN,4,13,NaN,NaN,NaN,NaN,3.5,0.0,3.5
29998,2,999800,10167.0,119.0,10166.0,57.0,NaN,NaN,10168,119,...,NaN,4,20,NaN,NaN,NaN,NaN,3.5,0.0,3.5


In [134]:
trading_dte = 8 - mk['day']
mk['time_to_expiry'] = 1 - mk['timestamp'] / (1_000_000 * trading_dte)

In [183]:
# df = get_first_two_dfs()
q = mk['day'] == 1
df = mk[q].copy()
df['time_to_expiry'] = 1 - df['timestamp'] / (1_000_000 * 7)

In [184]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['mid_price_rock'], name='Rock Mid Price'))
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['mid_price_voucher10000'], name='Rock Call Mid Price', yaxis='y2'))
fig.update_layout(title='Rock and Rock Call Prices over Timestamp', yaxis=dict(title='Rock Mid Price'), yaxis2=dict(title='Rock Call Bid Price', overlaying='y', side='right'))
fig.show()


In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

def black_scholes_call(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot / strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    call_price = (spot * norm.cdf(d1) - strike * norm.cdf(d2))
    return call_price

def black_scholes_put(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot / strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    put_price = (strike * norm.cdf(-d2) - spot * norm.cdf(-d1))
    return put_price

def delta(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.cdf(d1)

def gamma(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1)/(spot * volatility * np.sqrt(time_to_expiry))

def vega(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1) * (spot * np.sqrt(time_to_expiry)) / 100

def implied_volatility(call_price, spot, strike, time_to_expiry):
    # Define the equation where the root is the implied volatility
    def equation(volatility):
        estimated_price = black_scholes_call(spot, strike, time_to_expiry, volatility)
        return estimated_price - call_price

    # Using Brent's method to find the root of the equation
    implied_vol = brentq(equation, 1e-10, 3.0, xtol=1e-10)
    return implied_vol

def realized_vol(df_rock_call, window, step_size):
    df_rock_call[f'log_return_{step_size}'] = np.log(df_rock_call['mid_price_rock'].to_numpy()/df_rock_call['mid_price_rock'].shift(step_size).to_numpy())
    dt = step_size / 250 / 10000 
    # dt = step_size / 250 / 10000 
    df_rock_call[f'realized_vol_{step_size}'] = df_rock_call[f'log_return_{step_size}'].rolling(window=window).apply(lambda x: np.mean(x[::step_size]**2) / dt)
    df_rock_call[f'realized_vol_{step_size}'] = np.sqrt(df_rock_call[f'realized_vol_{step_size}'].to_numpy())
    return df_rock_call



In [186]:
# spot_price = 10000        # Spot price of the underlying asset
strike_price = 10000      # Strike price of the option
# call_price = 637.5         # Market price of the call option
# time_to_expiry = 1      # Time to expiry in years
# initial_guess = 16
# df['time_to_expiry'] = 1 - df['timestamp'] / 1000000 / 250
df['implied_vol'] = df.apply(lambda row: implied_volatility(row[f'mid_price_voucher{strike_price}'], row['mid_price_rock'], strike_price, row['time_to_expiry']), axis=1)
df['delta'] = df.apply(lambda row: delta(row['mid_price_rock'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)
df['gamma'] = df.apply(lambda row: gamma(row['mid_price_rock'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)
df['vega'] = df.apply(lambda row: vega(row['mid_price_rock'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)

In [187]:
df[['mid_price_rock', f'mid_price_voucher{strike_price}', 'implied_vol', 'delta', 'gamma', 'vega']].head(10)

,mid_price_rock,mid_price_voucher10000,implied_vol,delta,gamma,vega
10000,10516.0,517.5,0.023757,0.983401,0.000165,4.344013
10001,10515.0,516.5,0.023719,0.983375,0.000166,4.349271
10002,10515.0,516.5,0.023719,0.983375,0.000166,4.349239
10003,10517.0,519.5,0.025734,0.975679,0.000211,6.006137
10004,10519.5,521.5,0.024942,0.979478,0.000189,5.205629
10005,10517.5,519.5,0.024863,0.979417,0.000190,5.217862
10006,10515.5,517.5,0.024784,0.979354,0.000191,5.230167
10007,10516.0,517.5,0.023758,0.983401,0.000165,4.343796
10008,10515.5,517.5,0.024785,0.979354,0.000191,5.230092
10009,10517.0,518.0,0.022488,0.987870,0.000133,3.315545


In [188]:
spot_price = 10_517
strike_price = 10_000
time_to_expiry = 1

black_scholes_call(spot_price, strike_price, time_to_expiry, 0.023757)

np.float64(518.48338993158)

In [190]:
df['implied_vol'].mean()

np.float64(0.023052698192764183)

In [ ]:
dfc = df.copy()
realized_vol(dfc, 30, 1)['realized_vol_1']

,day,timestamp,bid_price_rock,bid_volume_rock,bid_price_2_rock,bid_volume_2_rock,bid_price_3_rock,bid_volume_3_rock,ask_price_rock,ask_volume_rock,...,mid_price_voucher10500,profit_and_loss_voucher10500,swmid_voucher10500,time_to_expiry,implied_vol,delta,gamma,vega,log_return_1,realized_vol_1
10000,1,0,10515.0,143.0,10514.0,57.0,NaN,NaN,10517,200,...,95.5,0.0,95.5,1.000000,0.023757,0.983401,0.000165,4.344013,NaN,NaN
10001,1,100,10514.0,104.0,10513.0,44.0,NaN,NaN,10516,104,...,94.5,0.0,94.5,0.999986,0.023719,0.983375,0.000166,4.349271,-0.000095,NaN
10002,1,200,10514.0,133.0,10513.0,62.0,NaN,NaN,10516,133,...,94.5,0.0,94.5,0.999971,0.023719,0.983375,0.000166,4.349239,0.000000,NaN
10003,1,300,10516.0,200.0,NaN,NaN,NaN,NaN,10518,149,...,95.5,0.0,95.5,0.999957,0.025734,0.975679,0.000211,6.006137,0.000190,NaN
10004,1,400,10519.0,108.0,10518.0,63.0,NaN,NaN,10520,103,...,97.5,0.0,97.5,0.999943,0.024942,0.979478,0.000189,5.205629,0.000238,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,999500,10224.0,162.0,NaN,NaN,NaN,NaN,10225,8,...,9.5,0.0,9.5,0.857214,0.021300,0.871947,0.001038,19.817346,0.000245,0.371418
19996,1,999600,10225.0,130.0,10224.0,67.0,NaN,NaN,10226,130,...,9.5,0.0,9.5,0.857200,0.020852,0.877936,0.001026,19.166962,0.000098,0.372490
19997,1,999700,10221.0,162.0,NaN,NaN,NaN,NaN,10223,108,...,9.5,0.0,9.5,0.857186,0.020886,0.873930,0.001048,19.598134,-0.000342,0.378863
19998,1,999800,10223.0,122.0,10222.0,65.0,NaN,NaN,10224,116,...,9.5,0.0,9.5,0.857171,0.021733,0.866201,0.001049,20.425199,0.000147,0.378866


In [206]:
np.log(df['mid_price_rock'] / df['mid_price_rock'].shift(1)).var() * (10000 * 365) ** (1/2)

np.float64(0.00012557409016140237)

In [136]:
import plotly.express as px

fig = px.line(df, x='timestamp', y='delta', title='Delta over Time')
fig.show()


ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['day', 'timestamp', 'bid_price_rock', 'bid_volume_rock', 'bid_price_2_rock', 'bid_volume_2_rock', 'bid_price_3_rock', 'bid_volume_3_rock', 'ask_price_rock', 'ask_volume_rock', 'ask_price_2_rock', 'ask_volume_2_rock', 'ask_price_3_rock', 'ask_volume_3_rock', 'mid_price_rock', 'profit_and_loss_rock', 'swmid_rock', 'bid_price_voucher9500', 'bid_volume_voucher9500', 'bid_price_2_voucher9500', 'bid_volume_2_voucher9500', 'bid_price_3_voucher9500', 'bid_volume_3_voucher9500', 'ask_price_voucher9500', 'ask_volume_voucher9500', 'ask_price_2_voucher9500', 'ask_volume_2_voucher9500', 'ask_price_3_voucher9500', 'ask_volume_3_voucher9500', 'mid_price_voucher9500', 'profit_and_loss_voucher9500', 'swmid_voucher9500', 'bid_price_voucher9750', 'bid_volume_voucher9750', 'bid_price_2_voucher9750', 'bid_volume_2_voucher9750', 'bid_price_3_voucher9750', 'bid_volume_3_voucher9750', 'ask_price_voucher9750', 'ask_volume_vo

In [78]:
fig = px.line(df, x='timestamp', y='vega', title='Vega over Time')
fig.show()

In [79]:
fig = px.line(df, x='timestamp', y='implied_vol', title='Implied Volatility over Time')
fig.show()


# backtest strat-trading implied volatility to mean

In [138]:
trading_dte = 8 - mk['day']
mk['time_to_expiry'] = 1 - mk['timestamp'] / (1_000_000 * trading_dte)

# q = mk['day'] == 0
# df = mk[q].copy()
df = mk.copy()

strike_price = 10000

for strike in [9500, 9750, 10000, 10250, 10500]:
    df[f'implied_vol_{strike}'] = df.apply(lambda row: implied_volatility(row[f'mid_price_voucher{strike}'], row['mid_price_rock'], strike, row['time_to_expiry']), axis=1)
    df[f'delta_{strike}'] = df.apply(lambda row: delta(row['mid_price_rock'], strike, row['time_to_expiry'], row[f'implied_vol_{strike}']), axis=1)
    df[f'gamma_{strike}'] = df.apply(lambda row: gamma(row['mid_price_rock'], strike, row['time_to_expiry'], row[f'implied_vol_{strike}']), axis=1)
    df[f'vega_{strike}'] = df.apply(lambda row: vega(row['mid_price_rock'], strike, row['time_to_expiry'], row[f'implied_vol_{strike}']), axis=1)

# df['implied_vol'] = df.apply(lambda row: implied_volatility(row[f'mid_price_voucher{strike_price}'], row['mid_price_rock'], strike_price, row['time_to_expiry']), axis=1)
# df['delta'] = df.apply(lambda row: delta(row['mid_price_rock'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)
# df['gamma'] = df.apply(lambda row: gamma(row['mid_price_rock'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)
# df['vega'] = df.apply(lambda row: vega(row['mid_price_rock'], strike_price, row['time_to_expiry'], row['implied_vol']), axis=1)

ValueError: f(a) and f(b) must have different signs

In [123]:
df.groupby('day')['implied_vol_10000'].describe()

,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
0,10000.0,0.025439,0.001029,0.021430,0.024720,0.025435,0.026127,0.029275
1,10000.0,0.023053,0.001232,0.019044,0.022156,0.023156,0.023952,0.026640
2,10000.0,0.019715,0.000704,0.016988,0.019238,0.019706,0.020178,0.023177


In [124]:
df.groupby('day')['implied_vol_10250'].describe()

,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
0,10000.0,0.023173,0.000322,0.022249,0.022936,0.023129,0.023380,0.024593
1,10000.0,0.021223,0.000314,0.020344,0.020971,0.021276,0.021463,0.022045
2,10000.0,0.019347,0.000385,0.018455,0.019074,0.019285,0.019607,0.020470


In [125]:
df.groupby('day')['implied_vol_10500'].describe()

,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
0,10000.0,0.022636,0.000403,0.021722,0.022201,0.022754,0.022968,0.023502
1,10000.0,0.021248,0.000508,0.020485,0.020877,0.021040,0.021507,0.023169
2,10000.0,0.020978,0.000847,0.019227,0.020298,0.020840,0.021600,0.023616


In [ ]:
df_backtest = df[['timestamp', f'mid_price_voucher{strike_price}', 'mid_price_rock', 'implied_vol', 'delta', 'vega']]

In [ ]:
df_backtest['implied_vol'].describe()

np.float64(0.02543853816937061)

In [91]:
# implied_vol_mean = 0.16
implied_vol_mean = 0.02543853816937061
implied_vol_std = df_backtest['implied_vol'].std()

In [107]:
import pandas as pd

# Set the threshold values

# upper_threshold = 0.006  # Threshold for selling option
# lower_threshold = -0.006  # Threshold for buying option
# close_threshold = 0.0001  # Threshold for clearing position
g = 3
c = 0.2
upper_threshold = g*implied_vol_std  # Threshold for selling option
lower_threshold = -g*implied_vol_std  # Threshold for buying option
close_threshold = c*implied_vol_std  # Threshold for clearing position

# Initialize variables
position = 0
pnl = 0
vega_pnl = 0
trade_history = []

# Iterate over each row in the dataframe
for idx, row in df_backtest.iterrows():
    implied_vol = row['implied_vol']
    if idx == 0:
        continue
    prev_implied_vol = df_backtest.iloc[idx-1]['implied_vol']
    mid_price_voucher = row[f'mid_price_voucher{strike_price}']
    mid_price_rock = row['mid_price_rock']
    vega = row['vega']
    d = row['delta']

    # Check if implied vol is above the upper threshold and no current position
    if implied_vol > implied_vol_mean + upper_threshold and position == 0:
        # Sell 1 delta hedged option
        position = -1
        entry_price_voucher = mid_price_voucher
        entry_price_rock = mid_price_rock
        trade_history.append((-1, entry_price_voucher, entry_price_rock, implied_vol))

    # Check if implied vol is below the lower threshold and no current position
    elif implied_vol < implied_vol_mean + lower_threshold and position == 0:
        # Buy 1 delta hedged option
        position = 1
        entry_price_voucher = mid_price_voucher
        entry_price_rock = mid_price_rock
        trade_history.append((1, entry_price_voucher, entry_price_rock, implied_vol))

    # Check if implied vol is within the close threshold and there is a current position
    elif abs(implied_vol - implied_vol_mean) <= close_threshold and position != 0:
        # Clear the position
        pnl += position * (mid_price_voucher - entry_price_voucher + d * (entry_price_rock - mid_price_rock))
        position = 0
        trade_history.append((0, mid_price_voucher, mid_price_rock, implied_vol))

    if position != 0:
        vega_pnl += position * vega * (implied_vol - prev_implied_vol) * 100
# Calculate final PnL if there is still an open position
if position != 0:
    pnl += position * (mid_price_voucher - entry_price_voucher + d * (entry_price_rock - mid_price_rock))

# Print the trade history and final PnL
print("Trade History:")
for trade in trade_history:
    print(f"Position: {trade[0]}, Option Price: {trade[1]}, Underlying Price: {trade[2]}, Implied Volatility: {trade[3]}")

print(f"\nFinal PnL: {pnl}")

Trade History:
Position: -1, Option Price: 515.5, Underlying Price: 10510.0, Implied Volatility: 0.02927544148101071
Position: 0, Option Price: 498.5, Underlying Price: 10495.5, Implied Volatility: 0.02562770446362991
Position: -1, Option Price: 513.5, Underlying Price: 10509.0, Implied Volatility: 0.028527562691845455
Position: 0, Option Price: 551.5, Underlying Price: 10550.0, Implied Volatility: 0.02540064673358045
Position: -1, Option Price: 545.5, Underlying Price: 10542.0, Implied Volatility: 0.028676497786131117
Position: 0, Option Price: 585.5, Underlying Price: 10584.5, Implied Volatility: 0.025289597327398276
Position: -1, Option Price: 579.5, Underlying Price: 10577.0, Implied Volatility: 0.028545074938557426
Position: 0, Option Price: 550.5, Underlying Price: 10549.0, Implied Volatility: 0.02537583570817812
Position: -1, Option Price: 554.5, Underlying Price: 10551.0, Implied Volatility: 0.029084960924291667
Position: 0, Option Price: 527.0, Underlying Price: 10525.0, Impli

In [108]:
vega_pnl * 600

np.float64(-14877.774274805664)

In [109]:
import pandas as pd

# Set the threshold values
upper_threshold = 0.005  # Threshold for selling option
lower_threshold = -0.005  # Threshold for buying option

# Initialize variables
position = 0
pnl = 0
trade_history = []

# Iterate over each row in the dataframe
for _, row in df_backtest.iterrows():
    implied_vol = row['implied_vol']
    mid_price_voucher = row['mid_price_voucher']
    mid_price_rock = row['mid_price_rock']
    d = row['delta']

    # Check if implied vol is above the upper threshold
    if implied_vol > implied_vol_mean + upper_threshold:
        # Sell to target position of -1
        if position > -1:
            quantity = -1 - position
            position = -1
            entry_price_voucher = mid_price_voucher
            entry_price_rock = mid_price_rock
            trade_history.append((quantity, entry_price_voucher, entry_price_rock, implied_vol))

    # Check if implied vol is below the lower threshold
    elif implied_vol < implied_vol_mean + lower_threshold:
        # Buy to target position of 1
        if position < 1:
            quantity = 1 - position
            position = 1
            entry_price_voucher = mid_price_voucher
            entry_price_rock = mid_price_rock
            trade_history.append((quantity, entry_price_voucher, entry_price_rock, implied_vol))

# Calculate final PnL for the remaining position
if position != 0:
    pnl += position * (mid_price_voucher - entry_price_voucher + d * (entry_price_rock - mid_price_rock))

# Print the trade history and final PnL
print("Trade History:")
for trade in trade_history:
    print(f"Quantity: {trade[0]}, Option Price: {trade[1]}, Underlying Price: {trade[2]}, Implied Volatility: {trade[3]}")

print(f"\nFinal PnL: {pnl}")

KeyError: 'mid_price_voucher'